In [17]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

N_EXAMPLES   = 20_000    # number of simulated data sets
N_PER_SAMPLE = 100       # points (x_i, y_i) per data set
SIGMA_EPS    = 0.5       # noise scale (known here)
THETA_RANGE  = (0.0, 10) # prior range for θ during training
BATCH_SIZE   = 256
EPOCHS       = 20


Generate the summary statistics data set to learn from, have a dataset of summary statistic theta pairs

In [18]:
def make_summary(x, y):
    """Return [S_xx, S_xy, n] for one data set."""
    S_xx = tf.reduce_sum(tf.square(x), axis=-1, keepdims=True)   # shape (batch, 1)
    S_xy = tf.reduce_sum(x * y,        axis=-1, keepdims=True)   # shape (batch, 1)

    # Broadcast n so it has shape (batch, 1) as well
    n_val = tf.cast(tf.shape(x)[-1], tf.float32)                 # scalar
    n     = tf.fill(tf.shape(S_xx), n_val)                       # shape (batch, 1)

    return tf.concat([S_xx, S_xy, n], axis=-1)                   # shape (batch, 3)


def simulate_batch(batch_size):
    """Generate a batch of independent data sets & their θ."""
    theta = tf.random.uniform([batch_size, 1], *THETA_RANGE)
    x     = tf.random.uniform([batch_size, N_PER_SAMPLE])
    noise = SIGMA_EPS * tf.random.normal([batch_size, N_PER_SAMPLE])
    y     = theta * x + noise
    summary = make_summary(x, y)                         # shape (batch, 3)
    return summary, theta

def make_dataset(n_examples):
    """tf.data.Dataset emitting (summary, θ) pairs. Allows for efficient processing"""
    steps = n_examples // BATCH_SIZE
    ds = tf.data.Dataset.range(steps).map(
        lambda _: simulate_batch(BATCH_SIZE),
        num_parallel_calls=tf.data.AUTOTUNE
    ).unbatch()
    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(int(N_EXAMPLES*0.9))
val_ds   = make_dataset(int(N_EXAMPLES*0.1))


Define our basic model, 2 dense layers with relu, 32 width

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)               # θ̂
])
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 32)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,217 (4.75 KB)

 Trainable params: 1,217 (4.75 KB)

 Non-trainable params: 0 (0.00 B)

Initialise and train our model

In [20]:
# Define our model
history = model.fit(train_ds.batch(BATCH_SIZE),
                    validation_data=val_ds.batch(BATCH_SIZE),
                    epochs=EPOCHS)


Epoch 1/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - loss: 146.1208 - val_loss: 0.9973
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - loss: 0.9841 - val_loss: 0.6982
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.6453 - val_loss: 0.5523
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - loss: 0.5398 - val_loss: 0.4512
Epoch 5/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - loss: 0.4770 - val_loss: 0.4259
Epoch 6/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 0.4175 - val_loss: 0.3748
Epoch 7/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - loss: 0.3927 - val_loss: 0.3243
Epoch 8/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step - loss: 0.3532 - val_loss: 0.3159
Epoch 9/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - loss: 0.3064 - val_loss: 0.3163
Epoch 10/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - loss: 0.2795 - val_loss: 0.2552
Epoch 11/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - loss: 0.2489 - val_loss: 0.2230
Epoch 12/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/st

Test to see how well performs on 1 data set.

In [21]:
# Simulate one brand-new data set
x_test  = np.random.rand(N_PER_SAMPLE).astype(np.float32)
theta_true = 5.0
y_test  = theta_true * x_test + SIGMA_EPS*np.random.randn(N_PER_SAMPLE).astype(np.float32)
summary_test = make_summary(x_test[None, :], y_test[None, :])  # shape (1,3)

theta_pred  = model.predict(summary_test)[0,0]
print(f"True θ = {theta_true:.2f},  Predicted θ = {theta_pred:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
True θ = 5.00,  Predicted θ = 4.88


Simulate a couple and find relative error

In [22]:
def average_relative_error(model, num_tests=1000):
    """
    Simulate `num_tests` new (x, y) samples, predict θ with the trained model,
    and return the mean relative error over the batch.
    """
    # Simulate a batch
    summaries, theta_true = simulate_batch(num_tests)        # (num_tests, 3) and (num_tests, 1)

    # Model prediction → shape (num_tests, 1)
    theta_pred = model.predict(summaries, verbose=0)

    # Relative error: |pred - true| / true   (add tiny eps to avoid div-by-0)
    eps       = 1e-8
    rel_error = np.abs(theta_pred.squeeze() - theta_true.numpy().squeeze()) / (theta_true.numpy().squeeze() + eps)

    # Return the mean error
    return rel_error.mean()

avg_re = average_relative_error(model, num_tests=1000)
print(f"Average relative error over 1000 tests: {avg_re:.4f}")


Average relative error over 1000 tests: 0.0730
